## Stream Processing Exercise 3 - Creating an Streaming job

Purpose of this exercise is to create your first Streaming job using Spark Streaming and reading from a Network socket.

First logon inside jupyter container with following command:
<pre>docker exec -it jupyter bash</pre>

Launch netcat server

<pre>nc -lkp 9999</pre>

Let’s code our Spark streaming job. First we import the names of the Spark Streaming classes and some implicit conversions from StreamingContext into our environment in order to add useful
methods to other classes we need (like DStream). StreamingContext is the main entry point for all streaming functionality. We create a local StreamingContext with two execution threads, and a batch interval of 5 seconds.

In [1]:
import sys

from pyspark import SparkContext
from pyspark.streaming import StreamingContext

sc = SparkContext(appName="PythonStreamingNetworkWordCount")

ssc = StreamingContext(sc, 5)


Using this context, we can create a DStream that represents streaming data from a TCP source, specified as hostname (e.g. localhost) and port (e.g. 9999).

In [2]:
lines = ssc.socketTextStream("localhost", 9999)

This lines DStream represents the stream of data that will be received from the data server. Each record in this DStream is a line of text. Next, we want to split the lines by space characters into words.

flatMap is a one-to-many DStream operation that creates a new DStream by generating multiple new records from each record in the source DStream. In this case, each line will be split into multiple words and the stream of words is represented as the words DStream. Next, we want to count these words.

In [3]:
counts = lines.flatMap(lambda line: line.split(" "))\
                  .map(lambda word: (word, 1))\
                  .reduceByKey(lambda a, b: a+b)
counts.pprint()


The words DStream is further mapped (one-to-one transformation) to a DStream of (word, 1) pairs, which is then reduced to get the frequency of words in each batch of data. Finally, wordCounts.print() will print a few of the counts generated every second.

Note that when these lines are executed, Spark Streaming only sets up the computation it will perform when it is started, and no real processing has started yet. To start the processing after all the transformations have been setup, we finally call

In [4]:
ssc.start()
ssc.awaitTermination()

-------------------------------------------
Time: 2021-01-03 21:07:00
-------------------------------------------

-------------------------------------------
Time: 2021-01-03 21:07:05
-------------------------------------------

-------------------------------------------
Time: 2021-01-03 21:07:10
-------------------------------------------
('hola', 2)

-------------------------------------------
Time: 2021-01-03 21:07:15
-------------------------------------------
('perro', 3)
('caracola', 1)

-------------------------------------------
Time: 2021-01-03 21:07:20
-------------------------------------------



KeyboardInterrupt: 